## Compte rendu R5.A.10, ANDRIAMISA Nayann, Groupe Nayru

#### Les installations :

In [ ]:
pip install redis

### Test du filtre bloom sur Redis

In [ ]:
import redis
import csv
from json import dumps
import time

In [ ]:
def perf_bloom(csv_file, n) :
    # Créer une connexion à Redis
    r = redis.Redis(host='localhost', port=6380, decode_responses=True)

    r.delete("bloom")
    r.execute_command("BF.RESERVE", "bloom", "0.01", "1700000")

    with open(csv_file, encoding='utf-8') as csvfile:
        my_reader = csv.DictReader(csvfile, delimiter='\t')
        my_data = [my_row for my_row in my_reader]

        pres = dup = 0
        print('Start to create the bloom filter over', n, 'inputs')
        st = time.process_time()

        for my_row in my_data[0:n]:
            if not r.execute_command("BF.EXISTS", "bloom", my_row['M']):
                r.execute_command("BF.ADD", "bloom", my_row['M'])
                pres = pres + 1
            else:
                dup = dup + 1

        et = time.process_time()
        res = et - st
        print('CPU Execution time:', res, 'seconds')
        print('We found', dup, 'duplicates in the input')

# Step 1
perf_bloom("DEMO.csv", 14000)

### set py pour des test de performances

In [ ]:
def perf_set(csv_file, n):

    r = redis.Redis(host='localhost', port=6380, decode_responses=True)

    r.delete("my_set")

    with open(csv_file, encoding = 'utf-8') as csvfile:
        my_reader = csv.DictReader(csvfile,delimiter='\t')
        my_data = [my_row for my_row in my_reader]
        #print(my_data)
        pres = dup = 0
        print('Start to create the set over',n,'inputs')
        # get the start time
        st = time.process_time()
        for my_row in my_data[0:n]:
            #print(my_row['M'])
            if not r.sismember("my_set", my_row['M']):
                r.sadd("my_set", my_row['M'])
                pres = pres + 1
            else:
                dup = dup + 1
        # get the end time
        et = time.process_time()
        # get execution time
        res = et - st
        print('CPU Execution time:', res, 'seconds')
        print('We found',dup,'duplicates in the input')
        print()
        print('Wall time (also known as clock time or wall-clock time) is simply the total time')
        print('elapsed during the measurement. Itâ€™s the time you can measure with a stopwatch.')
        print('It is the difference between the time at which a program finished its execution and')
        print('the time at which the program started. It also includes waiting time for resources.')
        print()
        print('CPU Time, on the other hand, refers to the time the CPU was busy processing')
        print('the programâ€™s instructions. The time spent waiting for other task to complete')
        print('(like I/O operations) is not included in the CPU time. It does not include')
        print('the waiting time for resources.')
#Step 1
 
perf_set("DEMO.csv", 1400)

### PUB/SUB avec Redis

In [ ]:
r = redis.Redis(host="localhost", port=6380, decode_responses=True)

jsonData = {
    "nom":"Andriamisa",
    "prenom":"Nayann"
}

jsonTest = json.dumps(jsonData)
r.publish('testPubSub', jsonTest)
print(r.get('testPubSub'))

In [ ]:
pip install schedule

In [ ]:
import json
import redis
import random
from datetime import datetime
import time

Publisher

In [ ]:
def generate_random_co2_value():
    return round(random.uniform(300, 1000))

while True:
    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    co2_value = generate_random_co2_value()
    
    # Créer un dictionnaire
    data = {'date': current_date, 'co2_value': co2_value}
    
    # Convertir le dictionnaire en format JSON
    json_data = json.dumps(data)
    
    r.publish('testPubSub', json_data)

    print(r.get('testPubSub'))
    
    subscriber = r.pubsub()

    subscriber.subscribe('testPubSub')

    for message in subscriber.listen():
        if message['type '] == 'message':
            print(f"Reçu : {message['data'].decode('utf-8')}")

    time.sleep(random.uniform(1, 3))

Subscriber

In [ ]:
import redis

subscriber = r.pubsub()

subscriber.subscribe('testPubSub')

for message in subscriber.listen():
    if message['type '] == 'message':
        print(f"Reçu : {message['data'].decode('utf-8')}")

In [ ]:
import redis
import schedule
import time

def func():
    global my_list
    #
    # Calcul de la moyenne
    #
    if len(my_list) > 0:
    	print("La moyenne des valeurs de CO2 sur la dernière minute est:", sum(my_list) / len(my_list))
    my_list.clear()

schedule.every(1).minutes.do(func)

r = redis.Redis(host='localhost', port=6380, db=0)

p = r.pubsub()
p.psubscribe('testPubSub')

my_list = []

first_message = True

while True:
    schedule.run_pending()
    m = p.get_message()
    #schedule.run_pending()
    if m != None:
       if not first_message:
          print('Message reçu:', m['data'])
          data_str = m['data'].decode('utf-8')  # Décoder les bytes en str
          data_dict = json.loads(data_str)  # Charger la chaîne JSON en un dictionnaire Python
          co2_value = data_dict.get('co2_value')
          if co2_value is not None:
                my_list.append(co2_value)

       else:
        print("Erreur : ça marche pas")
        first_message = False
        time.sleep(1)

In [ ]:
pip install pymongo

In [ ]:
import pymongo

# Connexion à MongoDB
client = pymongo.MongoClient("mongodb://192.168.0.5:27017/")

print(client)

db_name = "test"
db = client[db_name]
print(db)
